## Advanced Lane Finding Project

The goals / steps of this project are the following:

* ~~Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.~~
* ~~Apply a distortion correction to raw images.~~
* ~~Use color transforms, gradients, etc., to create a thresholded binary image.~~
* ~~Apply a perspective transform to rectify binary image ("birds-eye view").~~
* ~~Detect lane pixels and fit to find the lane boundary.~~
* ~~Determine the curvature of the lane and vehicle position with respect to center.~~
* ~~Warp the detected lane boundaries back onto the original image.~~
* ~~Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.~~

---
## First, I'll compute the camera calibration using chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [107]:
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./camera_cal/calibration*.jpg')
i = j = 0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    if ret == True:
        i += 1
        objpoints.append(objp)
        imgpoints.append(corners)
    j += 1

print('{} of {} images used successfully for calibration'.format(i,j))

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

17 of 20 images used successfully for calibration


### Test calibration

In [ ]:
test_distorted_img = cv2.imread('./camera_cal/calibration3.jpg')
test_corrected_img = cv2.undistort(test_distorted_img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (6,2))
ax1.imshow(test_distorted_img)
ax1.set_title('Distorted Image')
ax2.imshow(test_corrected_img)
ax2.set_title('Corrected Image')

# Get perspective transform for overhead view

In [ ]:
test_distorted_img = cv2.imread('./test_images/straight_lines1.jpg')
test_corrected_img = cv2.undistort(test_distorted_img, mtx, dist, None, mtx)
test_corrected_img = cv2.cvtColor(test_corrected_img, cv2.COLOR_RGB2BGR)

image_to_warp = test_corrected_img

input_vertices = np.float32([[520,500],[770, 500],[250, 680], [1050, 680]])
output_vertices = np.float32([[400,450],[940,450],[400, 700], [940,700]])

M = cv2.getPerspectiveTransform(input_vertices, output_vertices)
Minv = cv2.getPerspectiveTransform( output_vertices, input_vertices)
warped = cv2.warpPerspective(image_to_warp, M, (image_to_warp.shape[1], image_to_warp.shape[0]), flags=cv2.INTER_LINEAR)

In [ ]:
test_lanes_img = cv2.imread('./test_images/test1.jpg')
bgr_test_lanes_img = cv2.cvtColor(test_lanes_img, cv2.COLOR_RGB2BGR)
gray_test_lanes_img = cv2.cvtColor(test_lanes_img, cv2.COLOR_RGB2GRAY)
plt.imshow(bgr_test_lanes_img, cmap='gray')

In [ ]:
# CONSIDER WHETHER THERE ARE POTENTIAL BENEFITS TO ANOTHER CHOICE OF OPERATOR (ROTATIONAL SYMMETRY PROB NOT IMPORTANT)
# Sobel functions are returning 0,1... need to make them the same if you choose to use them again
def get_component_gradient_mask(image):
    sobelx = cv2.Sobel(gauss_img, cv2.CV_64F, 1, 0) #, ksize=sobel_kernel)
    sobely = cv2.Sobel(gauss_img, cv2.CV_64F, 0, 1)
    abs_sobel_x, abs_sobel_y = np.absolute(sobelx), np.absolute(sobely)
    #Convert the absolute value image to [0,1]:
    scaled_abs_sobel_x, scaled_abs_sobel_y = abs_sobel_x / np.max(abs_sobel_x), abs_sobel_y / np.max(abs_sobel_y)
    thresh_min_x, thresh_max_x, thresh_min_y, thresh_max_y = 0.1, 1.00, 0.35, 1.00
    sbinary = np.zeros_like(scaled_abs_sobel_x)
    sbinary[((scaled_abs_sobel_x >= thresh_min_x) & (scaled_abs_sobel_x <= thresh_max_x) == 1) & ((scaled_abs_sobel_y >= thresh_min_y) & (scaled_abs_sobel_y <= thresh_max_y) == 1)] = 1
    return sbinary

def get_magnitude_gradient_mask(image):
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0) #, ksize=sobel_kernel)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1)
    abs_sobel = np.sqrt(sobelx**2 + sobely**2)
    #Convert the absolute value image to [0,1]:
    scaled_abs_sobel = abs_sobel / np.max(abs_sobel)
    thresh_min, thresh_max = 0.5, 1.00 
    sbinary = np.zeros_like(scaled_abs_sobel)
    sbinary[(scaled_abs_sobel >= thresh_min) & (scaled_abs_sobel <= thresh_max)] = 1
    return sbinary

def get_direction_gradient_mask(image, kernel_size=3, threshold=(0.7, 1.2)):
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel_size)
    dir_sobel = np.arctan2(np.absolute(sobely), np.absolute(sobelx)) #should this be squared before or after scaling to (0,1)?
    binary_output =  np.zeros_like(dir_sobel)
    binary_output[(dir_sobel >= threshold[0]) & (dir_sobel <= threshold[1])] = 1
    return binary_output

def hsv_color_select(frame):
    # converts image to HSV to ease empirical selection 
    #  of color ranges for maskings
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
     
    lower_yellow = np.array([95,75,100], dtype=np.uint8)
    upper_yellow = np.array([105,255,255], dtype=np.uint8)

    lower_white = np.array([0,0,230], dtype=np.uint8)
    upper_white = np.array([180,35,255], dtype=np.uint8)

    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    white_mask = cv2.inRange(hsv_image, lower_white, upper_white)
    
    mask = cv2.bitwise_or(yellow_mask, white_mask)
    color_select = cv2.bitwise_and(frame, frame, mask=mask) # join to original image with bitwise-or
    return mask, color_select

def region_of_interest(image, vertices=None):
    ysize = image.shape[0]
    xsize = image.shape[1]
        
    bottom_left = np.array([0+(.10*xsize), ysize])
    bottom_right = np.array([xsize-(.10*xsize), ysize])
    apex = np.array([xsize/2, ysize*(5/10)])
    vertices = np.array([[bottom_left, bottom_right, apex]])
    #### THE ABOVE IS LESS GENERAL, AND SHOULD BE EXCLUDED OR GENRALIZED
    
    mask = np.zeros_like(image)   
    
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, np.int32(vertices), ignore_mask_color)
    
    masked_image = cv2.bitwise_and(image, mask)
    
    return masked_image

In [ ]:
def yield_poly_fit_lines(binary_warped):
    # Slightly modified Udacity implementation of histogramimg pixels, plotting polynomials, and returning curvature
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

    midpoint = np.int(histogram.shape[0]/2)
    leftx_base, rightx_base = np.argmax(histogram[:midpoint]), np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    window_height = np.int(binary_warped.shape[0]/nwindows)
    nonzero = binary_warped.nonzero()
    nonzeroy, nonzerox = np.array(nonzero[0]), np.array(nonzero[1])

    leftx_current, rightx_current = leftx_base, rightx_base

    margin = 100
    minpix = 50
    
    left_lane_inds, right_lane_inds = [], []

    for window in range(nwindows):
        win_y_low, win_y_high = binary_warped.shape[0] - (window+1)*window_height, binary_warped.shape[0] - window*window_height
        win_xleft_low, win_xleft_high = leftx_current - margin, leftx_current + margin
        win_xright_low, win_xright_high = rightx_current - margin, rightx_current + margin
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    left_lane_inds, right_lane_inds = np.concatenate(left_lane_inds), np.concatenate(right_lane_inds)

    leftx, lefty = nonzerox[left_lane_inds], nonzeroy[left_lane_inds]
    rightx, righty = nonzerox[right_lane_inds], nonzeroy[right_lane_inds]
    left_fit, right_fit = np.polyfit(lefty, leftx, 2), np.polyfit(righty, rightx, 2)
        
    y_eval = binary_warped.shape[0] * (2/3)
    ym_per_pix, xm_per_pix = 30/720, 3.7/700 
    
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    return (left_fit, right_fit), (left_curverad, right_curverad)

def draw_lanes_to_image(warped, original, left_fit, right_fit, left_curve, right_curve):
    # slightly modified Udacity implementation for line drawing, modified to add curvature
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    warp_zero = np.zeros_like(warped)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped.shape[1], warped.shape[0])) 
    
    result_image = cv2.addWeighted(original, 1, newwarp, 0.3, 0)
    message = """ ~RoC: {}""".format((left_curve + right_curve) / 2)
    cv2.putText(result_image, message, (50,100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
    return result_image

In [105]:
def image_pipeline(image):
    # Images require conversion, video doesn't
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    corrected_bgr_image = cv2.undistort(image, mtx, dist, None, mtx)

    # corrected_gray_image = cv2.cvtColor(distorted_image, cv2.COLOR_RGB2GRAY)
    # gray_test_lanes_img = gauss_img
    # gauss_img = cv2.GaussianBlur(gray_test_lanes_img, (3,3), 0)

    # dir_mask = get_direction_gradient_mask(gray_test_lanes_img)
    # mag_mask = get_magnitude_gradient_mask(gray_test_lanes_img)
    # comp_mask = get_component_gradient_mask(gray_test_lanes_img)
    # masked = np.logical_and(comp_mask, np.logical_and(mag_mask, dir_mask))
    # masked = np.logical_or(masked, hsv_mask)

    hsv_mask, hsv_selected = hsv_color_select(corrected_bgr_image)

    hsv_final = region_of_interest(hsv_mask)

    warped = cv2.warpPerspective(hsv_final, M, (hsv_final.shape[1], hsv_final.shape[0]), flags=cv2.INTER_LINEAR)
    
    (left_fit, right_fit), (left_curverad, right_curverad) = yield_poly_fit_lines(warped)
    warped = draw_lanes_to_image(warped, corrected_bgr_image, left_fit, right_fit, left_curverad, right_curverad)
    final = warped
    return warped


In [106]:
white_output = 'white.mp4'
clip1 = VideoFileClip("./project_video.mp4")
white_clip = clip1.fl_image(image_pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

1135.95873178
(array([ -2.64136130e-04,   3.57729437e-01,   3.32732156e+02]), array([ -1.44661193e-04,   2.99599944e-01,   8.53830233e+02])) (621.91316830381231, 1135.9587317762914)
[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4










  0%|          | 0/1261 [00:00<?, ?it/s]

1135.95873178
(array([ -2.64136130e-04,   3.57729437e-01,   3.32732156e+02]), array([ -1.44661193e-04,   2.99599944e-01,   8.53830233e+02])) (621.91316830381231, 1135.9587317762914)










  0%|          | 2/1261 [00:00<01:43, 12.12it/s]

819.243289144
(array([ -2.63665528e-04,   3.58448399e-01,   3.32892079e+02]), array([ -2.00593325e-04,   3.57622401e-01,   8.37411620e+02])) (623.02687354810575, 819.24328914374325)


...









100%|█████████▉| 1256/1261 [01:30<00:00, 11.60it/s]

1044.82867611
(array([  1.58850937e-05,  -3.53718493e-02,   4.87755805e+02]), array([  1.57185240e-04,  -1.15672059e-01,   1.02607315e+03])) (10338.518271931869, 1044.8286761070231)
1351.27550376
(array([ -6.10048114e-06,  -1.34336017e-02,   4.80572799e+02]), array([  1.21536180e-04,  -9.34598835e-02,   1.02192293e+03])) (26920.532666255967, 1351.2755037612831)
1649.59185055
(array([ -2.14792982e-05,   7.37001581e-03,   4.72633486e+02]), array([  9.95573607e-05,  -7.15710314e-02,   1.01403950e+03])) (7645.8467758667157, 1649.5918505495004)










100%|█████████▉| 1258/1261 [01:30<00:00, 12.25it/s]







100%|█████████▉| 1260/1261 [01:30<00:00, 12.77it/s]

133337.094415
(array([ -1.26989515e-05,   4.63424568e-03,   4.71486592e+02]), array([  1.23168535e-06,   2.45183116e-02,   9.86438965e+02])) (12932.323792820143, 133337.09441468609)
64804.687714
(array([  8.00844995e-06,   1.78300045e-03,   4.62869842e+02]), array([ -2.53420004e-06,   2.06502855e-02,   9.90858117e+02])) (20506.725183453942, 64804.68771404431)
9323.6726124
(array([ -2.09588494e-05,   2.76649790e-02,   4.55748628e+02]), array([  1.76143436e-05,   1.34164239e-02,   9.86856859e+02])) (7835.6854891985749, 9323.6726124038942)


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2min 27s, sys: 3.35 s, total: 2min 31s
Wall time: 1min 31s


In [ ]:
# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

* increase lower range of value to 150 only increased noise
* increase upper range of saturation to 30 improved broken white lanes on light street
* raising lower ranger of value to 90% allowed me to raise sat to 45 with no problem, but 35 was just as well